# Import

In [ ]:
import time
import os
import numpy as np
import tensorflow as tf
import scipy.io as sio
import pickle
import scipy.sparse as sp
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from jhyexp import my_KNN, my_Kmeans#, my_TSNE, my_Linear
from numpy import linalg as LA
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from models import GAT, HeteGAT, HeteGAT_multi
from utils import process

# Hyperparameteres

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
batch_size = 1
nb_epochs = 1
patience = 100
lr = 0.005  # learning rate
l2_coef = 0.001  # weight decay
# numbers of hidden units per each attention head in each layer
hid_units = [8]
n_heads = [8, 1]  # additional entry for the output layer
residual = False
nonlinearity = tf.nn.elu
model = HeteGAT_multi

print('----- Opt. hyperparams -----')
print('lr: ' + str(lr))
print('l2_coef: ' + str(l2_coef))
print('----- Archi. hyperparams -----')
print('nb. layers: ' + str(len(hid_units)))
print('nb. units per layer: ' + str(hid_units))
print('nb. attention heads: ' + str(n_heads))
print('residual: ' + str(residual))
print('nonlinearity: ' + str(nonlinearity))
print('model: ' + str(model))


# Load Data and Mask

In [ ]:
def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=bool)
  
def load_data_dblp(path):
    """
    Load the data 
    here is where we difine the meta-paths
    
    """
    with open(path, 'rb') as f:
      data = pickle.load(f)
    truelabels, truefeatures = data['labels'], data['features'].astype(float)
    N = truefeatures.shape[0]
    rownetworks = [np.maximum(data['PAP'] - np.eye(N), 0), np.maximum(data['PJP'] - np.eye(N), 0)]

    y = truelabels
    train_idx = data['train_idx']
    val_idx = data['val_idx']
    test_idx = data['test_idx']

    train_mask = sample_mask(train_idx, y.shape[0])
    val_mask = sample_mask(val_idx, y.shape[0])
    test_mask = sample_mask(test_idx, y.shape[0])

    y_train = np.zeros(y.shape)
    y_val = np.zeros(y.shape)
    y_test = np.zeros(y.shape)
    y_train[train_mask, :] = y[train_mask, :]
    y_val[val_mask, :] = y[val_mask, :]
    y_test[test_mask, :] = y[test_mask, :]

    truefeatures_list = [truefeatures, truefeatures]
    return rownetworks, truefeatures_list, y_train, y_val, y_test, train_mask, val_mask, test_mask

# Path import


In [ ]:
def custom_sort_key(file_name):
    numeric_part = file_name.split("_")[1]
    return int(numeric_part)

def import_data(path,data_name):
    """
    import name of file inside a folder
    path : The path to the folder wich cotains the Graph_t as file
    data_name : the name given to the data (ex:dblp,acm)
    
    """
    L_path = []
    data_path = []
    file_names = sorted(os.listdir(path), key=custom_sort_key)
    i = 0
    for fname in file_names:
        dname = data_name + str(i)
        fpath = os.path.join(path, fname)
        if os.path.isfile(fpath):
            
            name = path + '\\' + fname
            L_path.append(name)
        data_path.append(dname)
        i += 1
    return L_path,data_path
        


In [ ]:
path = r"C:\Users\JérômeTAM\Desktop\HAN\DBLP\DBLP GRAPH T=4"


L_path,data_path = import_data(path,'dblp')

In [ ]:
data_path

# Build Graph and HAN 

In [ ]:
def build_graph(model, fea_list,biases_list, y_train, y_val, y_test, train_mask, val_mask, test_mask,nb_nodes, ft_size,nb_classes,checkpt_file):
  with tf.Graph().as_default():
    with tf.name_scope('input'):
      ftr_in_list = [tf.compat.v1.placeholder(dtype=tf.float32,
                                              shape=(batch_size, nb_nodes, ft_size),
                                              name='ftr_in_{}'.format(i))
                    for i in range(len(fea_list))]

      bias_in_list = [tf.compat.v1.placeholder(dtype=tf.float32,
                                                shape=(batch_size, nb_nodes, nb_nodes),
                                                name='bias_in_{}'.format(i))
                      for i in range(len(biases_list))]

      lbl_in = tf.compat.v1.placeholder(dtype=tf.int32, 
                                        shape=(batch_size, nb_nodes, nb_classes),
                                        name='lbl_in')
      
      msk_in = tf.compat.v1.placeholder(dtype=tf.int32, 
                                        shape=(batch_size, nb_nodes),
                                        name='msk_in')
      
      attn_drop = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name='attn_drop')
      ffd_drop = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name='ffd_drop')
      is_train = tf.compat.v1.placeholder(dtype=tf.bool, shape=(), name='is_train')

    # forward
    logits, final_embedding, att_val = model.inference(ftr_in_list, nb_classes, nb_nodes, is_train,
                                                         attn_drop, ffd_drop,
                                                         bias_mat_list=bias_in_list,
                                                         hid_units=hid_units, n_heads=n_heads,
                                                         residual=residual, activation=nonlinearity)
    
    # cal masked_loss
    log_resh = tf.reshape(logits, [-1, nb_classes])
    lab_resh = tf.reshape(lbl_in, [-1, nb_classes])
    msk_resh = tf.reshape(msk_in, [-1])
    loss = model.masked_softmax_cross_entropy(log_resh, lab_resh, msk_resh)
    accuracy = model.masked_accuracy(log_resh, lab_resh, msk_resh)
    # optimzie
    train_op = model.training(loss, lr, l2_coef)

    saver = tf.compat.v1.train.Saver()

    init_op = tf.group(tf.compat.v1.global_variables_initializer(),
                       tf.compat.v1.local_variables_initializer())

    vlss_mn = np.inf
    vacc_mx = 0.0
    curr_step = 0

    with tf.compat.v1.Session(config=config) as sess:
      sess.run(init_op)

      train_loss_avg = 0
      train_acc_avg = 0
      val_loss_avg = 0
      val_acc_avg = 0

      for epoch in range(nb_epochs):
        tr_step = 0

        tr_size = fea_list[0].shape[0]

        # ================   training    ============

        while tr_step * batch_size < tr_size:
          fd1 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]
                 for i, d in zip(ftr_in_list, fea_list)}

          fd2 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]
                 for i, d in zip(bias_in_list, biases_list)}

          fd3 = {lbl_in: y_train[tr_step * batch_size:(tr_step + 1) * batch_size],
                 msk_in: train_mask[tr_step * batch_size:(tr_step + 1) * batch_size],
                 is_train: True,
                 attn_drop: 0.6,
                 ffd_drop: 0.6}

          fd = fd1
          fd.update(fd2)
          fd.update(fd3)
          _, loss_value_tr, acc_tr, att_val_train,train_final_embedding = sess.run([train_op, loss, accuracy, att_val,final_embedding],
                                                              feed_dict=fd)
          train_loss_avg += loss_value_tr
          train_acc_avg += acc_tr
          tr_step += 1


        vl_step = 0
        vl_size = fea_list[0].shape[0]
        

                      # =============   val       =================
        while vl_step * batch_size < vl_size:
          fd1 = {i: d[vl_step * batch_size:(vl_step + 1) * batch_size]
                for i, d in zip(ftr_in_list, fea_list)}


          fd2 = {i: d[vl_step * batch_size:(vl_step + 1) * batch_size]
                for i, d in zip(bias_in_list, biases_list)}

          fd3 = {lbl_in: y_val[vl_step * batch_size:(vl_step + 1) * batch_size],
                msk_in: val_mask[vl_step * batch_size:(vl_step + 1) * batch_size],
                is_train: False,
                attn_drop: 0.6,#0.0
                ffd_drop: 0.6}#0.0
          
          fd = fd1
          fd.update(fd2)
          fd.update(fd3)
          loss_value_vl, acc_vl,val_final_embedding = sess.run([loss, accuracy,final_embedding],
                                                 feed_dict=fd)
          val_loss_avg += loss_value_vl
          val_acc_avg += acc_vl
          vl_step += 1
            
                
                

        print('Epoch: {}, att_val: {}'.format(epoch, np.mean(att_val_train, axis=0)))
        print('Training: loss = %.5f, acc = %.5f | Val: loss = %.5f, acc = %.5f' %
                  (train_loss_avg / tr_step, train_acc_avg / tr_step,
                   val_loss_avg / vl_step, val_acc_avg / vl_step))

      

        train_loss_avg = 0
        train_acc_avg = 0
        val_loss_avg = 0
        val_acc_avg = 0
      saver.save(sess, checkpt_file)
      saver.restore(sess, checkpt_file)
      print('load model from : {}'.format(checkpt_file))
      ts_size = fea_list[0].shape[0]
      ts_step = 0
      ts_loss = 0.0
      ts_acc = 0.0
      while ts_step * batch_size < ts_size:
        fd1 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                   for i, d in zip(ftr_in_list, fea_list)}
        fd2 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                   for i, d in zip(bias_in_list, biases_list)}
        fd3 = {lbl_in: y_test[ts_step * batch_size:(ts_step + 1) * batch_size],
                   msk_in: test_mask[ts_step * batch_size:(ts_step + 1) * batch_size],
            
                   is_train: False,
                   attn_drop: 0.6,
                   ffd_drop: 0.6}

        fd = fd1
        fd.update(fd2)
        fd.update(fd3)
        loss_value_ts, acc_ts, jhy_final_embedding = sess.run([loss, accuracy, final_embedding],
                                                              feed_dict=fd)
        ts_loss += loss_value_ts
        ts_acc += acc_ts
        ts_step += 1
        
      
      sess.close()
      xx = np.expand_dims(jhy_final_embedding, axis=0)[test_mask]
      yy = y_test[test_mask]
  
  return jhy_final_embedding,xx,yy,test_mask,train_final_embedding,val_final_embedding
    

In [ ]:
def compile_all(data_path,L_path):
  Lx = []
  Ly = []
  test_emb = []
  Lt = []
  val_emb = []
  train_emb = []
  for dataset,path in zip(data_path,L_path):
    featype = 'fea'
    """
    replace r'C:\Users\JérômeTAM\Desktop\HAN\CHECKPOINT by path 
    """
    checkpt_file = r'C:\Users\JérômeTAM\Desktop\HAN\CHECKPOINT/{}/{}_allMP_multi_{}_.ckpt'.format(dataset, dataset, featype)
   

    adj_list, fea_list, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data_dblp(path)
    if featype == 'adj':
      fea_list = adj_list
    nb_nodes = fea_list[0].shape[0]
    ft_size = fea_list[0].shape[1]
    nb_classes = y_train.shape[1]

    fea_list = [fea[np.newaxis] for fea in fea_list]
    adj_list = [adj[np.newaxis] for adj in adj_list]
    y_train = y_train[np.newaxis]
    y_val = y_val[np.newaxis]
    y_test = y_test[np.newaxis]
    train_mask = train_mask[np.newaxis]
    val_mask = val_mask[np.newaxis]
    test_mask = test_mask[np.newaxis]

    biases_list = [process.adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list]

    emb,xx,yy,ts,v_emb,t_emb = build_graph(model, fea_list,biases_list, y_train, y_val, y_test, train_mask, val_mask, test_mask,nb_nodes, ft_size,nb_classes,checkpt_file)

    Lx.append(xx)
    Ly.append(yy)
    test_emb.append(emb)
    Lt.append(ts)
    val_emb.append(v_emb)
    train_emb.append(t_emb)
  return Lx,Ly,test_emb,Lt,val_emb,train_emb

In [ ]:
""""
xx : the list of nodes embedded for each Graph_t after HAN from the test set

yy : the list of true labels for test-set

test_mask : mask for test-set to use after we have the time aggregation

train_emb : train embedding for each Graph_t after HAN 

val_emb : validation embedding for each Graph_t after HAN 

test_emb : test embedding for each Graph_t after HAN 


"""
xx,yy,test_emb,test_mask,val_emb,train_emb = compile_all(data_path,L_path)

# Time Aggregation


In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self,nb_graph):
        super(MyModel, self).__init__()
        """
        Model with conv layer as deep neural network, Maybe can be optimize with another deep neural network or a custom deep neural network

        Chose between 1 conv for all Graph_t or x number of conv for x Graph_t

        Forward defines the operations

        Can be optimize with custom weight definition

        inputs : a list of Graph_i for i not equal to t 
        output : the predict Graph_t  

        
        """
        self.conv_layers1 = [tf.keras.layers.Conv1D(filters=64, kernel_size=1,activation=None,kernel_regularizer=tf.keras.regularizers.l1(0.1)) for i in range (nb_graph)]
        #self.conv_layers1 = tf.keras.layers.Conv1D(filters=64, kernel_size=1,activation='relu') 

    def forward(self, inputs):
        M = []
        G = []
        for i in range(len(inputs)):
            #x = self.conv_layers1(inputs[i])
            x = self.conv_layers1[i](inputs[i])
            #x = tf.reduce_sum(x, axis=0)
            #x = tf.nn.leaky_relu(x)
            x = tf.nn.softmax(tf.nn.leaky_relu(x))
            #x = tf.nn.softmax(x)
            #x = tf.reduce_sum(x, axis=0)
            M.append(x)
            
        for i in range(len(inputs)):
            G.append(M[i]*inputs[i])
        result = tf.reduce_sum(G, axis=0)

        return result

    def train(self, train_data, optimizer, loss):
        """
        Perform the backprobagation to learn the weight of the neural network
        """
        with tf.GradientTape() as tape:
            pred = self.forward(train_data[0])
            loss_value = loss(train_data[1], pred)
        grads = tape.gradient(loss_value, self.trainable_variables)
        optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss_value

    def evaluate(self, val_data, loss):
        """
        
        """
        pred = self.forward(val_data[0])
        loss_value = loss(val_data[1], pred)
        return loss_value
    
    def predict(self,pred_data):
        """
        """
        pred = self.forward(pred_data)
        return pred



In [ ]:
"""
Adding 1 dim to get shape (1,nb_nodes,agg_dim)

nb_nodes: number of nodes
agg_dim: dimension of aggregation from HAN 

"""
A_test = [test_emb[i][np.newaxis] for i in range(len(test_emb))]
A_train = [train_emb[i][np.newaxis] for i in range(len(train_emb))]
A_val = [val_emb[i][np.newaxis] for i in range(len(val_emb))]

In [ ]:
nb_graph = len(A_train[0:-1])
model_time = MyModel(nb_graph)
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.MeanSquaredError()

### Training

In [ ]:
for epoch in range(100):
    total_loss = 0.0
    train_loss = model_time.train([A_train[0:-1], A_train[-1]], optimizer, loss)
    total_loss += train_loss
    
    val_loss = model_time.evaluate([A_val[0:-1], A_val[-1]], loss)
    print(f"Epoch {epoch+1}: Train Loss = {total_loss}, Val Loss = {val_loss}")


### Test

In [ ]:
pred = model_time.predict(A_test[0:-1])
xx_t = pred.numpy()[test_mask[-1]]

In [ ]:
#Precision from HAN model 
my_KNN(xx[-1],yy[-1])

In [ ]:
#Precision from aggragated time 
my_KNN(xx_t,yy[-1])

## Prediction for graph5

In [ ]:
"""
Test with graph_5
"""
A_test_4 = A_test[0:4]
A_train_4 = A_train[0:4]
A_val_4 = A_val[0:4]
for i in range (5,len(A_test)):
    A_test_4.append(A_test[i])
    A_train_4.append(A_train[i])
    A_val_4.append(A_val[i])

In [ ]:
nb_graph = len(A_train_4)
model_time = MyModel(nb_graph)
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.MeanSquaredError()

In [ ]:
for epoch in range(100):
    total_loss = 0.0
    train_loss = model_time.train([A_train_4, A_train[4]], optimizer, loss)
    total_loss += train_loss
    
    val_loss = model_time.evaluate([A_val_4, A_val[4]], loss)
    print(f"Epoch {epoch+1}: Train Loss = {total_loss}, Val Loss = {val_loss}")

In [ ]:
pred = model_time.predict(A_test_4)
xx_t = pred.numpy()[test_mask[4]]

In [ ]:
#
my_KNN(xx[-1],yy[-1])

In [ ]:
#
my_KNN(xx_t,yy[-1])